In [4]:
import base64
import json
import pandas as pd
import numpy as np
import requests
import tensorflow as tf

In [3]:
df = pd.read_csv("data/Cleaned_HR_Data_Analysis.csv")

In [17]:
from pprint import PrettyPrinter
pp = PrettyPrinter()
pp.pprint(requests.get("https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model"))

<Response [200]>


In [26]:
CATEGORICAL_FEATURES = {
    "EmployeeStatus": 2,
    "EmployeeType": 3,
    "PayZone": 3,
    "EmployeeClassificationType": 3,
    "DepartmentType": 6,
    "GenderCode": 2,
    "RaceDesc": 5,
    "MaritalDesc": 4,
    "TrainingType": 2,
    "TrainingOutcome": 4
}

NUMERICAL_FEATURES = [
    "CurrentEmployeeRating",
    "EngagementScore",
    "SatisfactionScore",
    "Work-LifeBalanceScore",
    "TrainingDurationDays",
    "TrainingCost",
    "Age"
]
selected_features = list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES
inputs = df.loc[53, selected_features].to_dict()
inputs


{'EmployeeStatus': 'Active',
 'EmployeeType': 'Contract',
 'PayZone': 'Zone B',
 'EmployeeClassificationType': 'Full-Time',
 'DepartmentType': 'Sales',
 'GenderCode': 'Male',
 'RaceDesc': 'White',
 'MaritalDesc': 'Widowed',
 'TrainingType': 'Internal',
 'TrainingOutcome': 'Incomplete',
 'CurrentEmployeeRating': 5,
 'EngagementScore': 3,
 'SatisfactionScore': 4,
 'Work-LifeBalanceScore': 5,
 'TrainingDurationDays': 3,
 'TrainingCost': 442.56,
 'Age': 48}

In [27]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples":{
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
print(prediction)

None


In [28]:
response

<Response [400]>

In [ ]:
"""
0 (PIP) → Kinerja buruk, butuh perbaikan segera.
1 (Fully Meets) → Memenuhi harapan pekerjaan dengan baik.
2 (Exceeded) → Melebihi ekspektasi dalam kinerja.
3 (Need Improvement) → Performa kurang baik, tapi belum sampai tahap PIP.
"""